# Pandas Exercise: Jeopardy Dataset

First of all, I'll download the dataset from [Kaggle](https://www.kaggle.com/tunguz/200000-jeopardy-questions) using their dedicated API.

In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.dataset_download_files('tunguz/200000-jeopardy-questions', unzip=True)

Let's start by loading the .csv file into a pandas DataFrame. I'll import pandas and use its `read_csv` function.

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
df = pd.read_csv("JEOPARDY_CSV.csv")

Let's examine the file.

In [3]:
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,"No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves",Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,"The city of Yuma in this state has a record average of 4,055 hours of sunshine each year",Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", this company served its billionth burger",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Constitution of Mass., second President of the United States",John Adams


There seems to be a problem of empty spaces in the column names. Let's use `.split()` to fix it.

In [4]:
print(df.columns)

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')


In [5]:
df.rename(columns = {column_name : column_name.strip() for column_name in df.columns}, inplace = True)
print(df.columns)
df.head()

Index(['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question',
       'Answer'],
      dtype='object')


,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,"No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves",Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,"The city of Yuma in this state has a record average of 4,055 hours of sunshine each year",Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", this company served its billionth burger",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Constitution of Mass., second President of the United States",John Adams


Let's create a function that looks up certain questions based on a list of words. This function takes in a list of words and then returns only those rows where all words from the list are found in the "Question" column.
Please note that some fiddling was required in order to make the filter work regardless of capitalization (list comprehensions that use the `.lower()` function and, in the case of the question string, the `.split()` function).
The essential formula to check whether all the words in a list appear in a string is:
```
all([word in string for word in list])
```
The function `all()` returns a boolean based on whether all values in a list are True or not.

In [6]:
def filter_questions(wordlist):
    return df[df.apply(lambda row: all([
        word in [string.lower() for string in row["Question"].split(" ")] for word in [string.lower() for string in wordlist]
    ]), axis = 1)]

Let's test the function.

In [88]:
kingly_wordlist = ["king"]
kingly_jeopardy = filter_questions(kingly_wordlist)
print(kingly_jeopardy)

        Show Number    Air Date             Round                    Category  \
40             4680  2004-12-31  Double Jeopardy!  DR. SEUSS AT THE MULTIPLEX   
781            4335  2003-06-06         Jeopardy!                   MY PLACE?   
811            4335  2003-06-06  Double Jeopardy!                 "S"-OTERICA   
846            4541  2004-05-10         Jeopardy!                 U.S. STATES   
896            3834  2001-04-12         Jeopardy!          AIN'T THAT AMERICA   
...             ...         ...               ...                         ...   
216232         1251  1990-01-29         Jeopardy!                THANKS, GUYS   
216445         3644  2000-06-08   Final Jeopardy!                   THE BIBLE   
216612         2046  1993-06-28  Double Jeopardy!             BIBLICAL PEOPLE   
216752         5070  2006-09-29         Jeopardy!                 SIGNING OFF   
216777         5070  2006-09-29  Double Jeopardy!             ANCIENT HISTORY   

         Value  \
40      1

To help with the following statistical calculations, let's convert the values contained in the "Value" column into floats (they're currently strings).
Note that we need to fiddle with the data 

In [17]:
df["Value"] = df.apply(lambda row: float(row["Value"].strip("$").replace(",", "")) if row["Value"] != "None" else 0, axis = 1)
print(df.head())

   Show Number    Air Date      Round                         Category  Value  \
0         4680  2004-12-31  Jeopardy!                          HISTORY  200.0   
1         4680  2004-12-31  Jeopardy!  ESPN's TOP 10 ALL-TIME ATHLETES  200.0   
2         4680  2004-12-31  Jeopardy!      EVERYBODY TALKS ABOUT IT...  200.0   
3         4680  2004-12-31  Jeopardy!                 THE COMPANY LINE  200.0   
4         4680  2004-12-31  Jeopardy!              EPITAPHS & TRIBUTES  200.0   

                                                                                                      Question  \
0             For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory   
1  No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves   
2                     The city of Yuma in this state has a record average of 4,055 hours of sunshine each year   
3                         In 1963, live on "The Art Linkl

We can now check the average cost of questions associated with particular topics. Let's use a function to calculate that. We will add an if/else condition in case the function finds no questions associated with a particular topic.

In [52]:
def average_value_for_topics(wordlist):
    avg_value = filter_questions(wordlist)["Value"].mean()
    if pd.isna(avg_value):
        print("There are no questions about " + " and ".join(wordlist) + ".")
    else:
        print("The average value of questions concerning " + " and ".join(wordlist) + " is " + str(avg_value) + ".")
    return avg_value

Let's test the function.

In [54]:
average_value_for_topics(["king","england"])
average_value_for_topics(["walt disney"])
average_value_for_topics(["burger"])
average_value_for_topics(["country", "war"])

The average value of questions concerning king and england is 829.7297297297297.
There are no questions about walt disney.
The average value of questions concerning burger is 650.0.
The average value of questions concerning country and war is 853.0.


853.0

Next item: create a function that counts the count of unique answers in a given dataset. Simple enough!

In [85]:
unique_answers = lambda unique_answers : unique_answers.groupby("Answer")["Question"].nunique().sort_values(ascending=False)

Let's test the function with the whole dataset.

In [86]:
print(unique_answers(df).head())

Answer
China        214
Australia    213
Japan        194
Chicago      193
France       189
Name: Question, dtype: int64


Let's test the function with questions about "king."

In [87]:
print(unique_answers(filter_questions(["king"])))

Answer
Henry VIII     41
Solomon        23
Sweden         23
Norway         22
Richard III    21
               ..
India           1
Ireland         1
Isabella        1
Islam           1
zlati           1
Name: Question, Length: 1165, dtype: int64
